In [1]:
# Prolog - Auto Generated #
import os, uuid, matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot
import pandas

import sys
sys.tracebacklimit = 0

os.chdir(u'C:/Users/friha/PythonEditorWrapper_8de9bb44-c5e2-4b9b-a916-40b4add7c106')
dataset = pandas.read_csv('input_df_889eb8aa-9cdf-4f54-8631-b9d8d6f007b3.csv')

matplotlib.pyplot.figure(figsize=(5.55555555555556,4.16666666666667), dpi=72)
matplotlib.pyplot.show = lambda args=None,kw=None: matplotlib.pyplot.savefig(str(uuid.uuid1()))
# Original Script. Please update your script content here and once completed copy below section back to the original editing window #
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import re
import matplotlib.pyplot as plt # Standard import for Power BI Python visuals

# --- Helper function to sanitize strings for Patsy formulas ---
def sanitize_for_patsy(name):
    if not isinstance(name, str): name = str(name)
    name = re.sub(r'[^\w_]', '_', name)
    if name and name[0].isdigit(): name = 's_' + name
    if not name: return "sanitized_empty_val"
    return name

# --- Helper Function to Run Logistic Regression and Get Stats ---
def analyze_feature(df_input, outcome_col, original_feature_col_name,
                    original_reference_category,
                    group_name_display,
                    all_categories_ordered=None):
    results = []
    default_cols = {'log_OR': np.nan, 'p_value': np.nan,
                    'N_category': 0, 'N_worked_in_category': 0,
                    'N_total_analysis': 0, 'N_worked_total_analysis': 0,
                    'Percentage_worked_in_category': np.nan,
                    'Percentage_worked_total_analysis': np.nan}

    if original_feature_col_name not in df_input.columns:
        print(f"Warning: Feature column '{original_feature_col_name}' not found. Skipping {group_name_display}.")
        results.append({'CharacteristicGroup': group_name_display,
                        'CharacteristicValue': 'Feature column missing', **default_cols})
        return results

    df_feature = df_input[['user_id', outcome_col, original_feature_col_name]].copy()
    df_feature[outcome_col] = pd.to_numeric(df_feature[outcome_col], errors='coerce')
    df_feature[original_feature_col_name] = df_feature[original_feature_col_name].astype(str)
    df_feature.dropna(subset=[outcome_col, original_feature_col_name], inplace=True)

    if df_feature.empty:
        print(f"Info: No data for {group_name_display} after initial NaN drop. Skipping model.")
        error_char_value = 'No data after NaN drop'
        if all_categories_ordered: error_char_value = f"No data for {all_categories_ordered[0]} (and others)"
        results.append({'CharacteristicGroup': group_name_display,
                        'CharacteristicValue': error_char_value, **default_cols})
        return results

    try:
        df_feature[outcome_col] = df_feature[outcome_col].astype(np.int64)
    except Exception as e_astype:
        print(f"Error converting outcome '{outcome_col}' to np.int64 for {group_name_display}: {e_astype}. Skipping.")
        results.append({'CharacteristicGroup': group_name_display,
                        'CharacteristicValue': f'Outcome conversion error: {e_astype}', **default_cols})
        return results

    df_feature.drop_duplicates(subset=['user_id', original_feature_col_name, outcome_col], inplace=True)

    if all_categories_ordered is None:
        discovered_categories = sorted(df_feature[original_feature_col_name].unique().astype(str))
        categories_to_iterate_for_output = discovered_categories
        if not discovered_categories:
             print(f"Info: No unique categories found in data for '{original_feature_col_name}' in group '{group_name_display}'.")
             results.append({'CharacteristicGroup': group_name_display,
                             'CharacteristicValue': 'No categories in data', **default_cols})
             return results
    else:
        categories_to_iterate_for_output = [str(cat) for cat in all_categories_ordered]

    if original_feature_col_name == 'User Experience Group':
        print(f"DEBUG: Unique values for '{original_feature_col_name}' (group: {group_name_display}) being processed: {categories_to_iterate_for_output}")
        print(f"DEBUG: Value counts in filtered df_feature for '{original_feature_col_name}':\n{df_feature[original_feature_col_name].value_counts().head(10)}")

    sanitized_feature_col_name = sanitize_for_patsy(original_feature_col_name)
    df_feature[sanitized_feature_col_name] = df_feature[original_feature_col_name].apply(sanitize_for_patsy)
    sanitized_reference_category = sanitize_for_patsy(original_reference_category)

    if df_feature.empty or df_feature[outcome_col].nunique() < 2 or df_feature[sanitized_feature_col_name].nunique() < 1 :
        error_msg = 'Insufficient data or variance for modeling.'
        print(f"Info: {error_msg} for {group_name_display}. Outputting counts only.")
        for orig_cat_val in categories_to_iterate_for_output:
            N_cat = df_feature[df_feature[original_feature_col_name] == str(orig_cat_val)].shape[0]
            N_worked_cat = df_feature[(df_feature[original_feature_col_name] == str(orig_cat_val)) & (df_feature[outcome_col] == 1)].shape[0]
            perc_worked_cat = (N_worked_cat / N_cat * 100) if N_cat > 0 else np.nan
            total_n_analysis = df_feature.shape[0]
            total_worked_analysis = df_feature[outcome_col].sum() if total_n_analysis > 0 else 0
            results.append({
                'CharacteristicGroup': group_name_display, 'CharacteristicValue': str(orig_cat_val),
                'log_OR': np.nan, 'p_value': np.nan,
                'N_category': N_cat, 'N_worked_in_category': N_worked_cat,
                'N_total_analysis': total_n_analysis, 'N_worked_total_analysis': total_worked_analysis,
                'Percentage_worked_in_category': perc_worked_cat,
                'Percentage_worked_total_analysis': (total_worked_analysis / total_n_analysis * 100) if total_n_analysis > 0 else np.nan })
        return results

    actual_sanitized_categories_in_df = df_feature[sanitized_feature_col_name].unique()
    current_sanitized_ref = sanitized_reference_category
    if sanitized_reference_category not in actual_sanitized_categories_in_df:
        if df_feature[original_feature_col_name].mode().shape[0] > 0:
            mode_original_value = df_feature[original_feature_col_name].mode()[0]
            current_sanitized_ref = sanitize_for_patsy(mode_original_value)
            print(f"Warning: Sanitized reference '{sanitized_reference_category}' (from original '{original_reference_category}') not found for '{group_name_display}'. Using most frequent original value '{mode_original_value}' (sanitized: '{current_sanitized_ref}').")
            if current_sanitized_ref not in actual_sanitized_categories_in_df:
                 print(f"CRITICAL WARNING: Mode '{mode_original_value}' (sanitized: '{current_sanitized_ref}') also not in sanitized categories. Model will likely fail for {group_name_display}")
        else:
            print(f"CRITICAL: No mode available for reference for {group_name_display}. Cannot fit model reliably.")
            for orig_cat_val in categories_to_iterate_for_output: results.append({'CharacteristicGroup': group_name_display, 'CharacteristicValue': str(orig_cat_val), **default_cols, 'N_total_analysis': df_feature.shape[0]})
            return results
    sanitized_reference_category = current_sanitized_ref

    if df_feature[sanitized_feature_col_name].nunique() < 2:
        print(f"Info: Only one unique predictor category ('{df_feature[original_feature_col_name].unique()[0]}') for {group_name_display}. Outputting counts only.")
        for orig_cat_val in categories_to_iterate_for_output:
            N_cat = df_feature[df_feature[original_feature_col_name] == str(orig_cat_val)].shape[0]
            N_worked_cat = df_feature[(df_feature[original_feature_col_name] == str(orig_cat_val)) & (df_feature[outcome_col] == 1)].shape[0]
            perc_worked_cat = (N_worked_cat / N_cat * 100) if N_cat > 0 else np.nan
            total_n_analysis = df_feature.shape[0]
            total_worked_analysis = df_feature[outcome_col].sum() if total_n_analysis > 0 else 0
            results.append({
                'CharacteristicGroup': group_name_display, 'CharacteristicValue': str(orig_cat_val),
                'log_OR': np.nan, 'p_value': np.nan,
                'N_category': N_cat, 'N_worked_in_category': N_worked_cat,
                'N_total_analysis': total_n_analysis, 'N_worked_total_analysis': total_worked_analysis,
                'Percentage_worked_in_category': perc_worked_cat,
                'Percentage_worked_total_analysis': (total_worked_analysis / total_n_analysis * 100) if total_n_analysis > 0 else np.nan })
        return results

    formula = f"{outcome_col} ~ C({sanitized_feature_col_name}, Treatment(reference='{sanitized_reference_category}'))"
    print(f"Attempting to fit model for {group_name_display} with formula: {formula}")
    try:
        if not pd.api.types.is_numeric_dtype(df_feature[outcome_col]):
             raise ValueError(f"Outcome column '{outcome_col}' is not numeric for modeling.")
        model = smf.logit(formula, data=df_feature).fit(disp=0)
        log_odds = model.params
        p_values = model.pvalues
        for orig_cat_val_iter in categories_to_iterate_for_output:
            str_orig_cat_val = str(orig_cat_val_iter)
            sanitized_cat_val_iter = sanitize_for_patsy(str_orig_cat_val)
            N_cat = df_feature[df_feature[original_feature_col_name] == str_orig_cat_val].shape[0]
            N_worked_cat = df_feature[(df_feature[original_feature_col_name] == str_orig_cat_val) & (df_feature[outcome_col] == 1)].shape[0]
            perc_worked_cat = (N_worked_cat / N_cat * 100) if N_cat > 0 else np.nan
            total_n_analysis = df_feature.shape[0]
            total_worked_analysis = df_feature[outcome_col].sum()
            current_log_or_val, current_p_value_val = np.nan, np.nan
            if sanitized_cat_val_iter == sanitized_reference_category: pass
            else:
                param_name = f"C({sanitized_feature_col_name}, Treatment(reference='{sanitized_reference_category}'))[T.{sanitized_cat_val_iter}]"
                current_log_or_val = log_odds.get(param_name, np.nan)
                current_p_value_val = p_values.get(param_name, np.nan)
            results.append({
                'CharacteristicGroup': group_name_display, 'CharacteristicValue': str_orig_cat_val,
                'log_OR': current_log_or_val, 'p_value': current_p_value_val,
                'N_category': N_cat, 'N_worked_in_category': N_worked_cat,
                'N_total_analysis': total_n_analysis, 'N_worked_total_analysis': total_worked_analysis,
                'Percentage_worked_in_category': perc_worked_cat,
                'Percentage_worked_total_analysis': (total_worked_analysis / total_n_analysis * 100) if total_n_analysis > 0 else np.nan })
    except Exception as e:
        print(f"Error fitting model or processing results for {group_name_display} (feature: {original_feature_col_name}): {e}. Outputting counts only.")
        for orig_cat_val in categories_to_iterate_for_output:
            N_cat = df_feature[df_feature[original_feature_col_name] == str(orig_cat_val)].shape[0]
            N_worked_cat = df_feature[(df_feature[original_feature_col_name] == str(orig_cat_val)) & (df_feature[outcome_col] == 1)].shape[0]
            perc_worked_cat = (N_worked_cat / N_cat * 100) if N_cat > 0 else np.nan
            total_n_analysis = df_feature.shape[0]
            total_worked_analysis = df_feature[outcome_col].sum() if total_n_analysis > 0 else 0
            results.append({
                'CharacteristicGroup': group_name_display, 'CharacteristicValue': str(orig_cat_val),
                'log_OR': np.nan, 'p_value': np.nan,
                'N_category': N_cat, 'N_worked_in_category': N_worked_cat,
                'N_total_analysis': total_n_analysis, 'N_worked_total_analysis': total_worked_analysis,
                'Percentage_worked_in_category': perc_worked_cat,
                'Percentage_worked_total_analysis': (total_worked_analysis / total_n_analysis * 100) if total_n_analysis > 0 else np.nan })
    return results

# --- Main Script ---
all_results_list = []
outcome_variable_name = 'UserHasWorked'
default_error_row_values = {
    'log_OR': np.nan, 'p_value': np.nan, 'N_category': 0, 'N_worked_in_category': 0,
    'N_total_analysis': 0, 'N_worked_total_analysis': 0,
    'Percentage_worked_in_category': np.nan, 'Percentage_worked_total_analysis': np.nan }

if 'user_id' not in dataset.columns or outcome_variable_name not in dataset.columns:
    output_df = pd.DataFrame([{'CharacteristicGroup': 'Error', 'CharacteristicValue': f"'user_id' or '{outcome_variable_name}' missing.", **default_error_row_values}])
else:
    dataset[outcome_variable_name] = pd.to_numeric(dataset[outcome_variable_name], errors='coerce')
    dataset.dropna(subset=[outcome_variable_name], inplace=True)
    try:
        dataset[outcome_variable_name] = dataset[outcome_variable_name].astype(np.int64)
    except Exception as e_main_astype:
        print(f"CRITICAL ERROR in Main Script: Could not convert outcome variable '{outcome_variable_name}' to np.int64. Error: {e_main_astype}")
        output_df = pd.DataFrame([{'CharacteristicGroup': 'Error', 'CharacteristicValue': f"Failed to process outcome variable '{outcome_variable_name}'.", **default_error_row_values}])
    
    if 'output_df' not in locals(): # Only proceed if no critical error set output_df above
        experience_reference_category = 'Aucune expérience'
        all_results_list.extend(analyze_feature(dataset, outcome_variable_name, 'User Experience Group',
                                           original_reference_category=experience_reference_category,
                                           group_name_display='experience', all_categories_ordered=None))
        all_results_list.extend(analyze_feature(dataset, outcome_variable_name, 'User Has Min Wage Rate',
                                           original_reference_category=False,
                                           group_name_display='Rate - User Has Min Wage', all_categories_ordered=None))
        all_results_list.extend(analyze_feature(dataset, outcome_variable_name, 'User Has 15 Plus Rate',
                                           original_reference_category=False,
                                           group_name_display='Rate - User Has 15 Plus', all_categories_ordered=None))
        if 'User_Gender' in dataset.columns:
            gender_reference_category = 'F'
            all_results_list.extend(analyze_feature(dataset, outcome_variable_name, 'User_Gender',
                                               original_reference_category=gender_reference_category,
                                               group_name_display='Gender', all_categories_ordered=None))
        else:
            print("Info: 'User_Gender' column not found. Skipping.")
            all_results_list.append({'CharacteristicGroup': 'Gender', 'CharacteristicValue': 'Column not provided', **default_error_row_values})
        
        language_reference = 'NATIVE FRENCH'
        all_results_list.extend(analyze_feature(dataset, outcome_variable_name, 'User French Proficiency',
                                           original_reference_category=language_reference,
                                           group_name_display='Codage Langues', all_categories_ordered=None))
        
        if 'User Is In Paris' in dataset.columns and 'User Is In Bordeaux' in dataset.columns:
            temp_df_city = dataset[['user_id', outcome_variable_name, 'User Is In Paris', 'User Is In Bordeaux']].copy()
            def assign_city_group(row):
                is_paris, is_bordeaux = False, False
                if pd.notna(row['User Is In Paris']):
                    if isinstance(row['User Is In Paris'], (bool, np.bool_)): is_paris = row['User Is In Paris']
                    elif isinstance(row['User Is In Paris'], (int, float)): is_paris = bool(row['User Is In Paris'])
                    elif isinstance(row['User Is In Paris'], str): is_paris = row['User Is In Paris'].lower() == 'true'
                if pd.notna(row['User Is In Bordeaux']):
                    if isinstance(row['User Is In Bordeaux'], (bool, np.bool_)): is_bordeaux = row['User Is In Bordeaux']
                    elif isinstance(row['User Is In Bordeaux'], (int, float)): is_bordeaux = bool(row['User Is In Bordeaux'])
                    elif isinstance(row['User Is In Bordeaux'], str): is_bordeaux = row['User Is In Bordeaux'].lower() == 'true'
                if is_paris: return 'Paris'
                if is_bordeaux: return 'Bordeaux'
                return 'Other'
            temp_df_city['City_Group_Temp'] = temp_df_city.apply(assign_city_group, axis=1)
            df_paris_bordeaux = temp_df_city[temp_df_city['City_Group_Temp'].isin(['Paris', 'Bordeaux'])].copy()
            if not df_paris_bordeaux.empty and df_paris_bordeaux[outcome_variable_name].notna().all():
                 all_results_list.extend(analyze_feature(df_paris_bordeaux, outcome_variable_name, 'City_Group_Temp',
                                               original_reference_category='Bordeaux',
                                               group_name_display='City', all_categories_ordered=['Bordeaux', 'Paris']))
            else:
                print("Info: Not enough data for Paris/Bordeaux for City analysis or outcome variable has issues.")
                all_results_list.append({'CharacteristicGroup': 'City', 'CharacteristicValue': 'Paris/Bordeaux No Valid Data', **default_error_row_values})
        else:
            print("Info: City columns ('User Is In Paris', 'User Is In Bordeaux') not found.")
            all_results_list.append({'CharacteristicGroup': 'City', 'CharacteristicValue': 'Columns not provided', **default_error_row_values})

        # Final DataFrame creation
    if all_results_list:
        output_df = pd.DataFrame(all_results_list)
        cols_order = ['CharacteristicGroup', 'CharacteristicValue', 'log_OR', 'p_value',
                      'N_category', 'N_worked_in_category', 'Percentage_worked_in_category',
                      'N_total_analysis', 'N_worked_total_analysis', 'Percentage_worked_total_analysis']
        # Ensure only existing columns are selected, in case some analysis failed to produce all.
        output_df = output_df[[col for col in cols_order if col in output_df.columns]]

    elif 'output_df' not in locals(): # If no critical error set output_df and list is empty
        output_df = pd.DataFrame([{
            'CharacteristicGroup': 'Error',
            'CharacteristicValue': 'No results generated and no critical error caught.',
            **default_error_row_values
        }])

if 'output_df' not in locals():
    output_df = pd.DataFrame([{
        'CharacteristicGroup': 'Critical Script Failure',
        'CharacteristicValue': 'output_df was not created. Check logs.',
        **default_error_row_values
    }])

# --- Apply float formatting ---
# Columns for simple rounding to 5 decimal places (excluding p_value)
float_cols_to_round = [
    'log_OR',
    'Percentage_worked_in_category',
    'Percentage_worked_total_analysis'
]

for col in float_cols_to_round:
    if col in output_df.columns:
        # Ensure the column is numeric before rounding, coercing errors to NaN
        output_df[col] = pd.to_numeric(output_df[col], errors='coerce')
        output_df[col] = output_df[col].round(5)

# Custom formatting for the 'p_value' column
if 'p_value' in output_df.columns:
    # First, ensure p_value is numeric for comparison
    numeric_p_values = pd.to_numeric(output_df['p_value'], errors='coerce')

    # Define a function to apply the custom formatting
    def format_p_value_custom(p_val):
        if pd.isna(p_val):
            return ""  # Represent NaN as an empty string for cleaner display
        elif p_val < 0.001:  # This includes true zeros and very small numbers
            return "< 0.001"
        else:
            return f"{p_val:.5f}"  # Format other p-values to 5 decimal places as strings

    # Apply the custom formatting function
    output_df['p_value'] = numeric_p_values.apply(format_p_value_custom)

# Ensure output_df is the last expression.
output_df

DEBUG: Unique values for 'User Experience Group' (group: experience) being processed: ['Aucune expérience', 'Expérience <1', 'Expérience >10', 'Expérience [1-5]', 'Expérience [5-10]']
DEBUG: Value counts in filtered df_feature for 'User Experience Group':
User Experience Group
Expérience [1-5]     17423
Expérience <1         7592
Expérience [5-10]     5330
Expérience >10        5008
Aucune expérience     2234
Name: count, dtype: int64
Attempting to fit model for experience with formula: UserHasWorked ~ C(User_Experience_Group, Treatment(reference='Aucune_expérience'))
Attempting to fit model for Rate - User Has Min Wage with formula: UserHasWorked ~ C(User_Has_Min_Wage_Rate, Treatment(reference='False'))
Attempting to fit model for Rate - User Has 15 Plus with formula: UserHasWorked ~ C(User_Has_15_Plus_Rate, Treatment(reference='False'))
Info: 'User_Gender' column not found. Skipping.
Attempting to fit model for Codage Langues with formula: UserHasWorked ~ C(User_French_Proficiency, T

,CharacteristicGroup,CharacteristicValue,log_OR,p_value,N_category,N_worked_in_category,Percentage_worked_in_category,N_total_analysis,N_worked_total_analysis,Percentage_worked_total_analysis
0,experience,Aucune expérience,NaN,,2234,131,5.86392,37587,2912,7.74736
1,experience,Expérience <1,-0.54759,< 0.001,7592,264,3.47734,37587,2912,7.74736
2,experience,Expérience >10,0.34266,< 0.001,5008,404,8.06709,37587,2912,7.74736
3,experience,Expérience [1-5],0.44601,< 0.001,17423,1545,8.86759,37587,2912,7.74736
4,experience,Expérience [5-10],0.64962,< 0.001,5330,568,10.65666,37587,2912,7.74736
5,Rate - User Has Min Wage,False,NaN,,961,5,0.52029,37587,2912,7.74736
6,Rate - User Has Min Wage,True,2.80238,< 0.001,36626,2907,7.93698,37587,2912,7.74736
7,Rate - User Has 15 Plus,False,NaN,,30702,2058,6.70315,37587,2912,7.74736
8,Rate - User Has 15 Plus,True,0.67847,< 0.001,6885,854,12.40378,37587,2912,7.74736
9,Gender,Column not provided,NaN,,0,0,NaN,0,0,NaN


In [2]:
import plotly.graph_objects as go

headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'


df = output_df.copy()


desired_figure_width_pixels = 1500 

fig = go.Figure(data=[go.Table(
  columnwidth = [1.1, 1.1, 0.6, 0.6, 0.6, 1.2, 1.5, 1, 1.2, 1.8], # Specify column widths
  header=dict(
    values=list(df.columns),
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=df.values.transpose(),
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    autosize=False, # Must be False to manually set width/height
    width=desired_figure_width_pixels, # Set the total width of the figure
    # Dynamic height based on content: (number_of_rows + header_row) * cell_height + some_padding
    height=(len(df) + 1) * 25 + 60, # Example: 25px per cell, 60px for header and margins
    margin=dict(l=10, r=10, t=30, b=10) # Optional: adjust margins
)

fig.show()

In [ ]:
import plotly.io as pio  # For image export

import io # For handling image bytes in memory

img_bytes_variable = None # Initialize to check later if image was created
try:
    # Export to PNG image bytes.
    # `scale` parameter increases resolution (e.g., scale=2 is 2x resolution).
    # This requires the 'kaleido' package: pip install kaleido
    img_bytes_variable = pio.to_image(fig, format='png', scale=3) # Using scale=3 for higher res

    # Convert image bytes to a format Matplotlib can read
    image_stream = io.BytesIO(img_bytes_variable)
    img_for_plt = plt.imread(image_stream, format='png')

    # Display the image using Matplotlib
    # Adjust figsize for Matplotlib if needed, though it often scales well with imshow
    # plt.figure(figsize=(desired_figure_width_pixels/100, ((len(df_for_plotly) + 1) * 25 + 60)/100)) # Optional: control matplotlib fig size
    plt.imshow(img_for_plt)
    plt.axis('off')  # Hide axis ticks and labels
    plt.tight_layout(pad=0) # Make image use full figure area
    # plt.show() # This is the crucial line for Power BI to pick up the Matplotlib plot

except Exception as e:
    print(f"Error generating or displaying Plotly image: {e}")
    print("Kaleido might be missing (pip install kaleido) or another issue occurred.")
    # If image generation fails, Power BI should ideally pick up 'output_df' if it's the last expression
    # Ensure 'output_df' is defined from your previous script steps.

# --- Crucial part for Power BI ---
# If image generation was successful, plt.show() will make Power BI display it.
# If image generation failed, we want Power BI to fall back to rendering output_df.
if img_bytes_variable is not None:
    plt.show()
elif 'output_df' in locals(): # Check if output_df exists from your data processing
    print("Fallback: Displaying DataFrame as Power BI table.")
    output_df # Make output_df the last expression
else:
    print("Fallback: output_df not defined, and image generation failed.")

In [22]:
%pip install kaleido


[notice] A new release of pip is available: 24.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
